In [1]:
!pip3 install transformers sentencepiece

In [2]:
binary = False

# Carregando modelo treinado

In [3]:
model_path = "/content/drive/MyDrive/Mestrado/transformers/trained_models/ptt5_tweetsentbr/binary/checkpoint-1035/" if binary else \
  "/content/drive/MyDrive/Mestrado/transformers/trained_models/ptt5_tweetsentbr/all_classes/checkpoint-1385/"

In [4]:
model_path

'/content/drive/MyDrive/Mestrado/transformers/trained_models/ptt5_tweetsentbr/all_classes/checkpoint-1385/'

In [5]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [6]:
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/ptt5-base-portuguese-vocab")
model = T5ForConditionalGeneration.from_pretrained(model_path).eval()

# Definindo funções auxiliares para classificação

In [7]:
def prepare_input_from_text(text: str):

  return tokenizer(f"tweet: {text}", return_tensors="pt")

In [8]:
import torch

def run_classifier(text: str, skip_special_tokens: bool = False):

  inputs = prepare_input_from_text(text)

  with torch.no_grad():
    generated_ids = model.generate(**inputs, max_length=3, do_sample=False).squeeze()

  return tokenizer.decode(generated_ids, skip_special_tokens=skip_special_tokens).strip()

In [9]:
# text = "Frase de teste"
text = "Isto tem um gosto ruim"

In [10]:
a = prepare_input_from_text(text)

In [11]:
tokenizer.decode(a["input_ids"].squeeze())

'tweet: Isto tem um gosto ruim</s>'

In [12]:
out_str = run_classifier(text, True)
out_str

'negativo'

# Predição em tweets

In [13]:
import pandas as pd
from tqdm.notebook import tqdm

In [14]:
data_df = pd.read_csv("/content/drive/MyDrive/Mestrado/transformers/data/tweetsentbr_test.csv", index_col=0)

In [15]:
if binary:
  label2id = {
      "positivo": 0,
      "negativo": 1
  }
else:
  label2id = {
      "positivo": 1,
      "negativo": 2,
      "neutro": 0
  }

In [16]:
label2id

{'negativo': 2, 'neutro': 0, 'positivo': 1}

In [17]:
if binary:
  data_df = data_df[data_df["label"] != 0]
  data_df["label"] -= 1 # 0 será a classe positiva, 1 será a classe negativa

In [18]:
data_df.head()

,label,text
2403,1,Começou o The Noite com o Charlie Hunnam
1349,2,"No twitter a 3ª feira é chata,pois todo mundo ..."
5422,0,Bial declamando os do garçom Conversa Com Bial
1262,0,AI MEU DEUS É AGORA Master Chef BR
5438,2,A Miriam só não roda pq eu acho que o tal sorv...


In [19]:
predictions = []

for tweet in tqdm(data_df["text"], desc="- Classificando tweets"):
  predictions.append(run_classifier(tweet, True))

- Classificando tweets:   0%|          | 0/2463 [00:00<?, ?it/s]

In [20]:
data_df["prediction"] = predictions

In [21]:
data_df["prediction"] = data_df["prediction"].map(label2id)

In [22]:
data_df.head()

,label,text,prediction
2403,1,Começou o The Noite com o Charlie Hunnam,0
1349,2,"No twitter a 3ª feira é chata,pois todo mundo ...",2
5422,0,Bial declamando os do garçom Conversa Com Bial,2
1262,0,AI MEU DEUS É AGORA Master Chef BR,2
5438,2,A Miriam só não roda pq eu acho que o tal sorv...,2


In [23]:
suffix = "binary" if binary else "all_classes"
data_df.to_csv(f"/content/drive/MyDrive/Mestrado/transformers/data/predicoes/ptt5_{suffix}_preds.csv")

# Avaliação em termos de métricas de classificação

In [24]:
from sklearn.metrics import classification_report

In [25]:
real = data_df["label"].values
preds = data_df["prediction"].values

In [26]:
print(classification_report(real, preds))

              precision    recall  f1-score   support

           0       0.64      0.49      0.55       627
           1       0.78      0.83      0.80      1109
           2       0.69      0.75      0.72       727

    accuracy                           0.72      2463
   macro avg       0.70      0.69      0.69      2463
weighted avg       0.71      0.72      0.71      2463

